# Individual Identification Model Using Siamese Network

## 1. Initialization

In [0]:
%tensorflow_version 2.x
import tensorflow as tf

In [0]:
# Import libraries
import numpy as np
import cv2
import os
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop, Adam, Adamax

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as VGG16Pre
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input as XceptionPre
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as MNPre

In [3]:
# Mount Google Drive - Note this mounts your personal Google Drive to the directory stated
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Set up the location of the dataset to work with
path = os.path.join('/content/drive/My Drive/U C Berkeley - Darragh/')
train_path = os.path.join(path,'Training Data')
test_path = os.path.join(path,'Test Data')

## 2. Custom Model

### 2-1. Data Preprocessing

#### 2-1-1. Data Load
The following code loads all the images from the given path passed through the parameter `path` so that any dataset can be easily extracted by altering the parameter.

All the images are changed to grayscale, resized, converted to an array, and all the arrays of the images that belong to the same class of individuals are saved under the same key value in a dictionary, where the key value is a name of an individual (concatenation of species and individual names), and the values are a list of all the arrays of the images.

In [0]:
# Loads all the images grouped by individuals from the given directory path
def loadimgs(path):
    total_individuals = 1
    species_list = []
    data_dict = {}

    for species in os.listdir(path):
        species_list.append(species)
        print("loading species: " + species)
        species_path = os.path.join(path,species)
        for individual in os.listdir(species_path):
            individual_path = os.path.join(species_path, individual)            
            for filename in os.listdir(individual_path):
                image_path = os.path.join(individual_path, filename)
                image = cv2.imread(image_path)
                if image is not None:
                  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                  resized_image=cv2.resize(gray,(92,112),interpolation = cv2.INTER_AREA)
                  array_1d = np.asarray(resized_image)
                  if species+'-'+individual not in data_dict:
                    data_dict[species+'-'+individual] = []
                  data_dict[species+'-'+individual].append(array_1d)
                  total_individuals += 1
    return total_individuals, data_dict, species_list

In [6]:
# Loads both train and test dataset
print("---Loading Training Data...---")
train_size, train_data, train_species = loadimgs(train_path)
print("---Loading Test Data...---")
test_size, test_data, test_species = loadimgs(test_path)

---Loading Training Data...---
loading species: Amur Tiger
loading species: Bengal Tiger
loading species: Cheetah
loading species: Leopard
loading species: Lowland Tapir
loading species: Puma
loading species: White Rhino
loading species: Black Rhino
loading species: African lion
loading species: African elephant
loading species: Bongo
---Loading Test Data...---
loading species: Amur Tiger
loading species: Bengal Tiger
loading species: Cheetah
loading species: Leopard
loading species: Lowland Tapir
loading species: Puma
loading species: White Rhino
loading species: Black Rhino
loading species: African lion
loading species: African elephant
loading species: Bongo


In the following cells, a simple code is implemented to grasp a high-level understanding on the dataset.

In [7]:
# Train & Test data size
print("Training data size: ",train_size)
print("Test data size: ",test_size)

Training data size:  1724
Test data size:  205


In [8]:
# Number of species classes
print("Total number of species: ", len(train_species))
print(train_species)

Total number of species:  11
['Amur Tiger', 'Bengal Tiger', 'Cheetah', 'Leopard', 'Lowland Tapir', 'Puma', 'White Rhino', 'Black Rhino', 'African lion', 'African elephant', 'Bongo']


In [9]:
# Number of individual classes
print("Total number of classes: ", len(train_data.keys()))
print(train_data.keys())

Total number of classes:  98
dict_keys(['Amur Tiger-261', 'Amur Tiger-237', 'Amur Tiger-279', 'Amur Tiger-440', 'Amur Tiger-565', 'Amur Tiger-682', 'Amur Tiger-1020', 'Bengal Tiger-Aria', 'Bengal Tiger-Fenimore', 'Bengal Tiger-India', 'Bengal Tiger-Lucky', 'Bengal Tiger-Moki', 'Bengal Tiger-Mona', 'Bengal Tiger-Rajah', 'Bengal Tiger-Rajaji', 'Cheetah-Aiko', 'Cheetah-Alvin', 'Cheetah-Chiquita', 'Cheetah-Jamu', 'Cheetah-Kiki', 'Cheetah-Pano', 'Cheetah-Rusty', 'Cheetah-Sandy', 'Cheetah-Tearmark', 'Leopard-Keanu', 'Leopard-Lewa', 'Leopard-Mick', 'Leopard-Ombeli', 'Leopard-Timbila', 'Leopard-Tony', 'Leopard-Wahoo', 'Leopard-Shakira', 'Lowland Tapir-Sorocaba 2', 'Lowland Tapir-Chuva F', 'Lowland Tapir-Sorocaba 5', 'Lowland Tapir-Edinha F', 'Lowland Tapir-Feminha F', 'Lowland Tapir-Pistolinha M', 'Lowland Tapir-Sorocaba', 'Lowland Tapir-Chuvisco M', 'Lowland Tapir-Riscado M', 'Puma-M-Taz', 'Puma-M-Skit', 'Puma-M-Pops', 'Puma-M-Phoenix', 'Puma-M-Oldex', 'Puma-M-Juvboy', 'Puma-M-Darby', 'Puma-F

In [10]:
# Size of an array of one sample image
train_data["Amur Tiger-261"][0].shape

(112, 92)

#### 2-1-2. Generating Training Dataset
The following function `get_data` generates an input data to be fed into the Siamese network later. 

The key concept of the Siamese network is that a model predicts whether or not two given input images belong to the same class. In order to train the model, both images of the same class and different classes need to be given so that the model can differentiate different characteristics between classes.

The function `get_data` exists for that reason. It generates the same number of same individual pairs(denoted by `x_genuine_pair`) and different individual pairs(denoted by `x_imposite_pair`). It also generates the corresponding binary labels - 0 if the two images are from two different individuals; 1 if the two images are from the same individual.

I would like to note that the pairs of images are extracted from the same species as the individual identification model will use the predicted species from the species classification model as one of its input.

In [0]:
def get_data(size, total_sample_size, dataset):
  image = dataset["Amur Tiger-261"][0]
  image = image[::size, ::size]
  dim1 = image.shape[0]
  dim2 = image.shape[1]
    
  # Initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
  x_genuine_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])  # 2 is for pairs
  y_genuine = np.zeros([total_sample_size, 1])

  x_imposite_pair = np.zeros([total_sample_size, 2, 1, dim1, dim2])
  y_imposite = np.zeros([total_sample_size, 1])

  species_dict = {}
  individuals = list(dataset.keys())

  # Generates all possible pairs of the two images within the same species
  for species in train_species:

    # Filter only individuals within the same species
    individuals_new = [ind for ind in individuals if ind.find(species) != -1]
    #print(species, individuals_new)

    # Same individual pairs
    count1 = 0
    print("Generating same individual pairs for "+species)
    for ind in individuals_new:
      footprints = dataset[ind]
      max_idx = len(footprints) - 1
      for idx, img in enumerate(footprints):
        counter = idx + 1
        while counter <= max_idx:
          img1 = img
          img2 = footprints[counter]
          # Reduce the size
          img1 = img1[::size, ::size]
          img2 = img2[::size, ::size]
          # Store the images to the initialized numpy array
          x_genuine_pair[count1, 0, 0, :, :] = img1
          x_genuine_pair[count1, 1, 0, :, :] = img2
          # Assign the label as one as we are drawing images from the same individual (genuine pair)
          y_genuine[count1] = 1
          counter += 1
          count1 += 1

    # Different individual pairs
    count2 = 0
    print("Generating different individual pairs for "+species)
    for idx, img in enumerate(individuals_new[:-1]):
      ind1 = individuals_new[idx]
      footprints1 = dataset[ind1]
      ind2_list = individuals_new[idx+1:]
      for idx2, img2 in enumerate(ind2_list):
        ind2 = ind2_list[idx2]
        footprints2 = dataset[ind2]
        #print(ind1, ind2)
        for fp1 in footprints1:
          for fp2 in footprints2:
            img1 = fp1
            img2 = fp2
            # Reduce the size
            img1 = img1[::size, ::size]
            img2 = img2[::size, ::size]
            # Store the images to the initialized numpy array
            x_imposite_pair[count2, 0, 0, :, :] = img1
            x_imposite_pair[count2, 1, 0, :, :] = img2
            # Assign the label as zero as we are drawing images from different individuals
            y_imposite[count2] = 0
            count2 += 1
  
  # Generate the SAME NUMBER of pairs for two target classes (0: different individuals, 1: same individuals)
  count = min(count1, count2)

  x_genuine_pair_new = np.zeros([count, 2, 1, dim1, dim2])  # 2 is for pairs
  y_genuine_new = np.zeros([count, 1])

  x_imposite_pair_new = np.zeros([count, 2, 1, dim1, dim2])
  y_imposite_new = np.zeros([count, 1])

  genuine_idx = np.random.choice(range(count1), count, replace=False)
  imposite_idx = np.random.choice(range(count2), count, replace=False)

  for idx1, idx2, counter in zip(genuine_idx, imposite_idx, range(count)):
    x_genuine_pair_new[counter, 0, 0, :, :] = x_genuine_pair[idx1, 0, 0, :, :]
    x_genuine_pair_new[counter, 1, 0, :, :] = x_genuine_pair[idx1, 1, 0, :, :]
    y_genuine_new[counter] = 1

    x_imposite_pair_new[counter, 0, 0, :, :] = x_imposite_pair[idx2, 0, 0, :, :]
    x_imposite_pair_new[counter, 1, 0, :, :] = x_imposite_pair[idx2, 1, 0, :, :]
    y_imposite_new[counter] = 0

  # Concatenate genuine pairs and imposite pairs to get the whole dataset
  X = np.concatenate([x_genuine_pair_new, x_imposite_pair_new], axis=0)/255
  Y = np.concatenate([y_genuine_new, y_imposite_new], axis=0)
  print("The End")
  return X, Y

In [12]:
size = 2
total_sample_size = 50000
X, Y = get_data(size, total_sample_size, train_data)
print(len(X), len(Y))

Generating same individual pairs for Amur Tiger
Generating different individual pairs for Amur Tiger
Generating same individual pairs for Bengal Tiger
Generating different individual pairs for Bengal Tiger
Generating same individual pairs for Cheetah
Generating different individual pairs for Cheetah
Generating same individual pairs for Leopard
Generating different individual pairs for Leopard
Generating same individual pairs for Lowland Tapir
Generating different individual pairs for Lowland Tapir
Generating same individual pairs for Puma
Generating different individual pairs for Puma
Generating same individual pairs for White Rhino
Generating different individual pairs for White Rhino
Generating same individual pairs for Black Rhino
Generating different individual pairs for Black Rhino
Generating same individual pairs for African lion
Generating different individual pairs for African lion
Generating same individual pairs for African elephant
Generating different individual pairs for A

#### 2-1-3. Data Split
The final step in data preprocessing before the modeling is to split the dataset into train and test(validation) dataset. 75:25 data split ratio is applied to the dataset - 75% of the dataset is used to train the model, and the remaining 25% is used to validate the model in the end.

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.25)

In [14]:
x_train.shape

(597, 2, 1, 56, 46)

In [15]:
y_train.shape

(597, 1)

In [16]:
x_test.shape

(199, 2, 1, 56, 46)

In [17]:
y_test.shape

(199, 1)

### 2-2. Model Development

#### 2-2-1. Base Model
In the following code, a base Siamese network model is built. The model is composed of two convolutional layers with rectified linear unit (ReLU) activations and maxpooling followed by a flat layer.

In [0]:
def build_base_network(input_shape):
  seq = Sequential()
    
  nb_filter = [6, 12]
  kernel_size = 3
    
  # Convolutional layer 1
  seq.add(Convolution2D(nb_filter[0], kernel_size, kernel_size, input_shape=input_shape, padding='valid', data_format="channels_first")) #NHWC
  seq.add(Activation('relu'))
  seq.add(MaxPooling2D(pool_size=(2, 2)))  
  seq.add(Dropout(.25))
    
  # Convolutional layer 2
  seq.add(Convolution2D(nb_filter[1], kernel_size, kernel_size, padding='valid', data_format="channels_first"))
  seq.add(Activation('relu'))
  seq.add(MaxPooling2D(pool_size=(2, 2), data_format="channels_first")) # Keras1 > dim_ordering='th'
  seq.add(Dropout(.25))

  # Flatten 
  seq.add(Flatten())
  seq.add(Dense(128, activation='relu'))
  seq.add(Dropout(0.1))
  seq.add(Dense(50, activation='relu'))

  seq.summary()
  
  return seq

Next, the image pairs are fed into the base network, which returns embeddings, that is, feature vectors - `feat_vecs_a` and `feat_vecs_b`.

In [0]:
input_dim = x_train.shape[2:]
img_a = Input(shape=input_dim)
img_b = Input(shape=input_dim)

In [20]:
print(input_dim)

(1, 56, 46)


In [21]:
base_network = build_base_network(input_dim)
feat_vecs_a = base_network(img_a)
feat_vecs_b = base_network(img_b)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 6, 18, 15)         60        
_________________________________________________________________
activation (Activation)      (None, 6, 18, 15)         0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 9, 15)          0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 9, 15)          0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 12, 3, 5)          336       
_________________________________________________________________
activation_1 (Activation)    (None, 12, 3, 5)          0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 1, 2)          0

#### 2-2-2. Model Training
The following code trains the base model defined above with the optimal hyperparameters that were identified in our multiple trials. Some examples of the hyperparameters that were altered to enhance the model are optimizer, loss function, and number of epochs.

The feature vectors from the base model are passed to the energy function to compute the distance between them, which uses Euclidean distance as a function.

In [0]:
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

In [0]:
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])

In [0]:
# Set up number of epochs and optimizer for our model
epochs = 200
optimizer = Adam(0.005)

In [0]:
model = Model(inputs=[img_a, img_b], outputs=distance)

Next, `contrastive_loss` function is defined to be used as the loss function of the model.

In [0]:
def contrastive_loss(y_true, y_pred):
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

In [0]:
model.compile(loss=contrastive_loss, optimizer=optimizer)

In [0]:
img_1 = x_train[:, 0]
img_2 = x_train[:, 1]

In [29]:
model.fit([img_1, img_2], y_train, validation_split=.25, batch_size=64, verbose=2, epochs=epochs)

Epoch 1/200
7/7 - 0s - loss: 0.2644 - val_loss: 0.4131
Epoch 2/200
7/7 - 0s - loss: 0.2613 - val_loss: 0.4221
Epoch 3/200
7/7 - 0s - loss: 0.2600 - val_loss: 0.4023
Epoch 4/200
7/7 - 0s - loss: 0.2585 - val_loss: 0.3963
Epoch 5/200
7/7 - 0s - loss: 0.2581 - val_loss: 0.3901
Epoch 6/200
7/7 - 0s - loss: 0.2576 - val_loss: 0.3851
Epoch 7/200
7/7 - 0s - loss: 0.2579 - val_loss: 0.3824
Epoch 8/200
7/7 - 0s - loss: 0.2580 - val_loss: 0.3658
Epoch 9/200
7/7 - 0s - loss: 0.2486 - val_loss: 0.3599
Epoch 10/200
7/7 - 0s - loss: 0.2461 - val_loss: 0.3404
Epoch 11/200
7/7 - 0s - loss: 0.2527 - val_loss: 0.3435
Epoch 12/200
7/7 - 0s - loss: 0.2493 - val_loss: 0.3175
Epoch 13/200
7/7 - 0s - loss: 0.2548 - val_loss: 0.3216
Epoch 14/200
7/7 - 0s - loss: 0.2468 - val_loss: 0.3174
Epoch 15/200
7/7 - 0s - loss: 0.2391 - val_loss: 0.2925
Epoch 16/200
7/7 - 0s - loss: 0.2284 - val_loss: 0.2872
Epoch 17/200
7/7 - 0s - loss: 0.2254 - val_loss: 0.2745
Epoch 18/200
7/7 - 0s - loss: 0.2229 - val_loss: 0.2484
E

#### 2-2-3. Model Evaluation
The trained model is then evaluated based on the validation dataset that was separated out of the training dataset in data preprocessing. The model is evaluated by measuring its accuracy.

*Please note that this step does not evaluate the model based on the test dataset and that it measures the accuracy based on the pairwise matching of the images - that is, if the given two images belong to the same class or not - not based on whether or not the model correctly predicts individual classes.*

In [0]:
pred = model.predict([x_test[:, 0], x_test[:, 1]])

In [0]:
def compute_accuracy(predictions, labels):
  return labels[predictions.ravel() < 0.5].mean()

In [32]:
compute_accuracy(pred, y_test)

0.7333333333333333

The following code saves the weights of the trained model so that the model can be easily reused without having to train the model again.

In [0]:
# Save model
# Purposely commented out the below code to prevent the currently using weights from getting overwritten
model.save_weights("/content/drive/My Drive/U C Berkeley - Darragh/csv/siamese_custom_model.h5")

The same model evaluation is performed using the weights loaded in the Google Drive in the above step.

In [0]:
trained_model = Model(inputs=[img_a, img_b], outputs=distance)
trained_model.compile(loss=contrastive_loss, optimizer=optimizer)
trained_model.load_weights("/content/drive/My Drive/U C Berkeley - Darragh/csv/siamese_custom_model.h5")

In [0]:
predicted_values = trained_model.predict([x_test[:, 0], x_test[:, 1]])

In [36]:
compute_accuracy(predicted_values, y_test)

0.7333333333333333

## 3. Pretrained Model
The same series of steps in Section 2 are taken below. The only difference between the previous section and this section is that the previous section uses a custom model, whereas this section uses a pretrained model - in this case, VGG16.

Explanation of the below codes are omitted for the reason stated above except when there is any difference that needs to be noted.

### 3-1. Data Preprocessing

#### 3-1-1. Data Load

In [37]:
# Initial model configuration setting based on the value given in the variable "BASE_MODEL"
BASE_MODEL="vgg16"  #Choices are: vgg16, mobilenetv2, xception

if BASE_MODEL=='vgg16':
  train_imagefile="Training-Images-224.csv"
  train_labelfile="Training-Labels-224.txt"
  test_imagefile="Test-Images-224.csv"
  test_labelfile="Test-Labels-224.txt"
  input_shape=(56,56,3) #(224,224,3)
  pretrained_model='species_classification_vgg16_model.h5'
  preprocessor=VGG16Pre
  savefile='vgg16_best_model'
  savemodel='vgg16_best_model.h5'
  zero_model=VGG16(weights='imagenet',include_top=False,input_shape=input_shape,)
elif BASE_MODEL=="mobilenetv2":
  train_imagefile="Train-Images-Mobile-224.csv"
  train_labelfile="Train-Labels-Mobile-224.txt"
  test_imagefile="Test-Images-Mobile-224.csv"
  test_labelfile="Test-Labels-Mobile-224.txt"
  input_shape=(224,224,3)
  pretrained_model='species_classification_mobilenetv2_model.h5'
  preprocessor=MNPre
  savefile='mobilenetv2_best_model'
  zero_model=MobileNetV2(weights='imagenet',include_top=False,input_shape=input_shape)
elif BASE_MODEL=="xception":
  train_imagefile="Training-Images-Xception-224.csv"
  train_labelfile="Training-Labels-Xception-224.txt"
  test_imagefile="Test-Images-Xception-224.csv"
  test_labelfile="Test-Labels-Xception-224.txt"
  input_shape=(224,224,3)
  pretrained_model='species_classification_xception_model.h5'
  preprocessor=XceptionPre
  savefile='xception_best_model'
  zero_model=Xception(weights='imagenet',include_top=False,input_shape=input_shape)

58892288/58889256 [==============================] - 1s 0us/step


The only difference in the following `loadimgs2` function is that the below function does not change the images to grayscale and resizes the images to a different size, compared to `loadimgs` function in Section 2.

In [0]:
def loadimgs2(path):
    total_individuals = 1
    species_list = []
    data_dict = {}

    for species in os.listdir(path):
        species_list.append(species)
        print("loading species: " + species)
        species_path = os.path.join(path,species)
        for individual in os.listdir(species_path):
            individual_path = os.path.join(species_path, individual)
            for filename in os.listdir(individual_path):
                image_path = os.path.join(individual_path, filename)
                image = cv2.imread(image_path)
                if image is not None:
                  #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
                  resized_image=cv2.resize(image,(112,112),interpolation = cv2.INTER_AREA) #92,112
                  array_1d = np.asarray(resized_image)
                  if species+'-'+individual not in data_dict:
                    data_dict[species+'-'+individual] = []
                  data_dict[species+'-'+individual].append(array_1d)
                  total_individuals += 1
    return total_individuals, data_dict, species_list

In [39]:
print("---Loading Training Data...---")
train_size, train_data, train_species = loadimgs2(train_path)
print("---Loading Test Data...---")
test_size, test_data, test_species = loadimgs2(test_path)

---Loading Training Data...---
loading species: Amur Tiger
loading species: Bengal Tiger
loading species: Cheetah
loading species: Leopard
loading species: Lowland Tapir
loading species: Puma
loading species: White Rhino
loading species: Black Rhino
loading species: African lion
loading species: African elephant
loading species: Bongo
---Loading Test Data...---
loading species: Amur Tiger
loading species: Bengal Tiger
loading species: Cheetah
loading species: Leopard
loading species: Lowland Tapir
loading species: Puma
loading species: White Rhino
loading species: Black Rhino
loading species: African lion
loading species: African elephant
loading species: Bongo


In [40]:
train_data["Amur Tiger-261"][0].shape

(112, 112, 3)

In [41]:
train_data["Amur Tiger-261"][0][::size, ::size, ::].shape

(56, 56, 3)

#### 3-1-2. Generating Training Dataset
The only difference in the following function from the the corresponding function in Section 2 is that `get_data2` function creates the final numpy arrays with different shapes because images are not converted to grayscale in this step - images shapes take a form of (X, X, 3) instead of (X, X).

In [0]:
def get_data2(size, total_sample_size, dataset):
  image = dataset["Amur Tiger-261"][0]
  image = image[::size, ::size, ::]
  dim1 = image.shape[0]
  dim2 = image.shape[1]
    
  # Initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
  x_genuine_pair = np.zeros([total_sample_size, 2, dim1, dim2, 3])  # 2 is for pairs
  y_genuine = np.zeros([total_sample_size, 1])

  x_imposite_pair = np.zeros([total_sample_size, 2, dim1, dim2, 3])
  y_imposite = np.zeros([total_sample_size, 1])

  species_dict = {}
  individuals = list(dataset.keys())

  # Generates all possible pairs of the two images within the same species
  for species in train_species:

    # Filter only individuals within the same species
    individuals_new = [ind for ind in individuals if ind.find(species) != -1]

    # Same individual pairs
    count1 = 0
    print("Generating same individual pairs for "+species)
    for ind in individuals_new:
      footprints = dataset[ind]
      max_idx = len(footprints) - 1
      for idx, img in enumerate(footprints):
        counter = idx + 1
        while counter <= max_idx:
          img1 = img
          img2 = footprints[counter]
          # Reduce the size
          img1 = img1[::size, ::size, ::]
          img2 = img2[::size, ::size, ::]
          # Store the images to the initialized numpy array
          x_genuine_pair[count1, 0, :, :, :] = img1
          x_genuine_pair[count1, 1, :, :, :] = img2
          # Assign the label as one as we are drawing images from the same individual (genuine pair)
          y_genuine[count1] = 1
          counter += 1
          count1 += 1

    # Different individual pairs
    count2 = 0
    print("Generating different individual pairs for "+species)
    for idx, img in enumerate(individuals_new[:-1]):
      ind1 = individuals_new[idx]
      footprints1 = dataset[ind1]
      ind2_list = individuals_new[idx+1:]
      for idx2, img2 in enumerate(ind2_list):
        ind2 = ind2_list[idx2]
        footprints2 = dataset[ind2]
        #print(ind1, ind2)
        for fp1 in footprints1:
          for fp2 in footprints2:
            img1 = fp1
            img2 = fp2
            # Reduce the size
            img1 = img1[::size, ::size, ::]
            img2 = img2[::size, ::size, ::]
            # Store the images to the initialized numpy array
            x_imposite_pair[count2, 0, :, :, :] = img1
            x_imposite_pair[count2, 1, :, :, :] = img2
            # Assign the label as zero as we are drawing images from different individuals
            y_imposite[count2] = 0
            count2 += 1
  
  # Generate the same number of pairs for two target classes (0: different individuals, 1: same individuals)
  count = min(count1, count2)

  x_genuine_pair_new = np.zeros([count, 2, dim1, dim2, 3])  # 2 is for pairs
  y_genuine_new = np.zeros([count, 1])

  x_imposite_pair_new = np.zeros([count, 2, dim1, dim2, 3])
  y_imposite_new = np.zeros([count, 1])

  genuine_idx = np.random.choice(range(count1), count, replace=False)
  imposite_idx = np.random.choice(range(count2), count, replace=False)

  for idx1, idx2, counter in zip(genuine_idx, imposite_idx, range(count)):
    x_genuine_pair_new[counter, 0, :, :, :] = x_genuine_pair[idx1, 0, :, :, :]
    x_genuine_pair_new[counter, 1, :, :, :] = x_genuine_pair[idx1, 1, :, :, :]
    y_genuine_new[counter] = 1

    x_imposite_pair_new[counter, 0, :, :, :] = x_imposite_pair[idx2, 0, :, :, :]
    x_imposite_pair_new[counter, 1, :, :, :] = x_imposite_pair[idx2, 1, :, :, :]
    y_imposite_new[counter] = 0

  # Concatenate genuine pairs and imposite pairs to get the whole data
  X = np.concatenate([x_genuine_pair_new, x_imposite_pair_new], axis=0)/255
  Y = np.concatenate([y_genuine_new, y_imposite_new], axis=0)
  print("The End")
  return X, Y

In [43]:
size = 2
total_sample_size = 50000
X, Y = get_data2(size, total_sample_size, train_data)
print(len(X), len(Y))

Generating same individual pairs for Amur Tiger
Generating different individual pairs for Amur Tiger
Generating same individual pairs for Bengal Tiger
Generating different individual pairs for Bengal Tiger
Generating same individual pairs for Cheetah
Generating different individual pairs for Cheetah
Generating same individual pairs for Leopard
Generating different individual pairs for Leopard
Generating same individual pairs for Lowland Tapir
Generating different individual pairs for Lowland Tapir
Generating same individual pairs for Puma
Generating different individual pairs for Puma
Generating same individual pairs for White Rhino
Generating different individual pairs for White Rhino
Generating same individual pairs for Black Rhino
Generating different individual pairs for Black Rhino
Generating same individual pairs for African lion
Generating different individual pairs for African lion
Generating same individual pairs for African elephant
Generating different individual pairs for A

#### 3-1-3. Data Split

In [0]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=.25)

In [45]:
x_train.shape

(597, 2, 56, 56, 3)

### 3-2. Model Development

#### 3-2-1. Base Model

In [0]:
img_1 = x_train[:, 0]
img_2 = x_train[:, 1]

csvpath='/content/drive/My Drive/U C Berkeley - Darragh/csv'

In [47]:
img_1.shape

(597, 56, 56, 3)

In [0]:
input_shape = (56, 56, 3)
left_input=Input(input_shape)
right_input=Input(input_shape)

In [0]:
def createSiameseNetwork(input_shape):
  model = Sequential()
  model.add(zero_model)

  # Flatten
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.1))
  model.add(Dense(50, activation='relu'))
  
  return model

In [0]:
base_network = createSiameseNetwork((56, 56, 3))
feat_vecs_a = base_network(left_input)
feat_vecs_b = base_network(right_input)
distance = Lambda(euclidean_distance, output_shape=eucl_dist_output_shape)([feat_vecs_a, feat_vecs_b])
    
# Connect the inputs with the outputs
siamese = Model(inputs=[left_input,right_input],outputs=distance)

In [51]:
siamese.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            [(None, 56, 56, 3)]  0                                            
__________________________________________________________________________________________________
sequential_1 (Sequential)       (None, 50)           14786802    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 1)            0           sequential_1[1][0]         

#### 3-2-2. Model Training

In [0]:
epochs = 50

In [0]:
siamese.compile(loss=contrastive_loss, optimizer=Adam(lr = 0.0005))

In [54]:
siamese.fit([img_1, img_2], y_train, validation_split=.25, batch_size=64, verbose=2, epochs=epochs)

Epoch 1/50
7/7 - 2s - loss: 21.1536 - val_loss: 0.4452
Epoch 2/50
7/7 - 1s - loss: 0.2972 - val_loss: 0.4060
Epoch 3/50
7/7 - 1s - loss: 0.2729 - val_loss: 0.3515
Epoch 4/50
7/7 - 1s - loss: 0.2518 - val_loss: 0.2721
Epoch 5/50
7/7 - 1s - loss: 0.2299 - val_loss: 0.2674
Epoch 6/50
7/7 - 1s - loss: 0.2249 - val_loss: 0.2622
Epoch 7/50
7/7 - 1s - loss: 0.2312 - val_loss: 0.2067
Epoch 8/50
7/7 - 1s - loss: 0.2063 - val_loss: 0.2015
Epoch 9/50
7/7 - 1s - loss: 0.1951 - val_loss: 0.1948
Epoch 10/50
7/7 - 1s - loss: 0.1853 - val_loss: 0.1873
Epoch 11/50
7/7 - 1s - loss: 0.1676 - val_loss: 0.1759
Epoch 12/50
7/7 - 1s - loss: 0.1450 - val_loss: 0.1620
Epoch 13/50
7/7 - 1s - loss: 0.1553 - val_loss: 0.1499
Epoch 14/50
7/7 - 1s - loss: 0.1828 - val_loss: 0.1650
Epoch 15/50
7/7 - 1s - loss: 0.1521 - val_loss: 0.1722
Epoch 16/50
7/7 - 1s - loss: 0.1424 - val_loss: 0.1765
Epoch 17/50
7/7 - 1s - loss: 0.1540 - val_loss: 0.1330
Epoch 18/50
7/7 - 1s - loss: 0.1317 - val_loss: 0.1234
Epoch 19/50
7/7 - 

#### 3-2-3. Model Evaluation

In [0]:
pred = siamese.predict([x_test[:, 0], x_test[:, 1]])

In [56]:
compute_accuracy(pred, y_test)

0.912621359223301

In [0]:
siamese.save_weights("/content/drive/My Drive/U C Berkeley - Darragh/csv/siamese_vgg16_model.h5")

## 4. Model Evaluation with True Test Dataset
The following code evaluates the model based on test dataset instead of validation dataset. Also, instead of measuring the accuracy based on pairwise matching prediction, it measure the true accuracy - that is, how well the model predicts individuals.

In [58]:
print(test_species)

['Amur Tiger', 'Bengal Tiger', 'Cheetah', 'Leopard', 'Lowland Tapir', 'Puma', 'White Rhino', 'Black Rhino', 'African lion', 'African elephant', 'Bongo']


In [59]:
print(test_data.keys())

dict_keys(['Amur Tiger-237', 'Amur Tiger-261', 'Amur Tiger-279', 'Amur Tiger-440', 'Amur Tiger-565', 'Amur Tiger-682', 'Amur Tiger-1020', 'Bengal Tiger-Aria', 'Bengal Tiger-Fenimore', 'Bengal Tiger-India', 'Bengal Tiger-Lucky', 'Bengal Tiger-Moki', 'Bengal Tiger-Mona', 'Bengal Tiger-Rajah', 'Bengal Tiger-Rajaji', 'Cheetah-Alvin', 'Cheetah-Aiko', 'Cheetah-Chiquita', 'Cheetah-Tearmark', 'Cheetah-Jamu', 'Cheetah-Kiki', 'Cheetah-Rusty', 'Cheetah-Sandy', 'Cheetah-Pano', 'Leopard-Keanu', 'Leopard-Shakira', 'Leopard-Lewa', 'Leopard-Mick', 'Leopard-Tony', 'Leopard-Timbila', 'Leopard-Wahoo', 'Leopard-Ombeli', 'Lowland Tapir-Chuva F', 'Lowland Tapir-Chuvisco M', 'Lowland Tapir-Edinha F', 'Lowland Tapir-Feminha F', 'Lowland Tapir-Pistolinha M', 'Lowland Tapir-Riscado M', 'Lowland Tapir-Sorocaba', 'Lowland Tapir-Sorocaba 2', 'Lowland Tapir-Sorocaba 5', 'Puma-F-Archback', 'Puma-F-Cassie', 'Puma-F-Lip', 'Puma-F-Spots', 'Puma-M-Darby', 'Puma-M-Juvboy', 'Puma-M-Oldex', 'Puma-M-Phoenix', 'Puma-M-Pops',

In [0]:
# Load trained model - VGG16
trained_model = Model(inputs=[left_input, right_input], outputs=distance)
trained_model.compile(loss=contrastive_loss, optimizer='adam', metrics=['accuracy'])
trained_model.load_weights("/content/drive/My Drive/U C Berkeley - Darragh/csv/siamese_vgg16_model.h5")

In [61]:
species_dict = {}
for species in test_species:
  for individual in train_data.keys():
    if species in individual:
      if species not in species_dict:
        species_dict[species] = [individual]
      else:
        species_dict[species].append(individual)
        
print(species_dict)

{'Amur Tiger': ['Amur Tiger-261', 'Amur Tiger-237', 'Amur Tiger-279', 'Amur Tiger-440', 'Amur Tiger-565', 'Amur Tiger-682', 'Amur Tiger-1020'], 'Bengal Tiger': ['Bengal Tiger-Aria', 'Bengal Tiger-Fenimore', 'Bengal Tiger-India', 'Bengal Tiger-Lucky', 'Bengal Tiger-Moki', 'Bengal Tiger-Mona', 'Bengal Tiger-Rajah', 'Bengal Tiger-Rajaji'], 'Cheetah': ['Cheetah-Aiko', 'Cheetah-Alvin', 'Cheetah-Chiquita', 'Cheetah-Jamu', 'Cheetah-Kiki', 'Cheetah-Pano', 'Cheetah-Rusty', 'Cheetah-Sandy', 'Cheetah-Tearmark'], 'Leopard': ['Leopard-Keanu', 'Leopard-Lewa', 'Leopard-Mick', 'Leopard-Ombeli', 'Leopard-Timbila', 'Leopard-Tony', 'Leopard-Wahoo', 'Leopard-Shakira'], 'Lowland Tapir': ['Lowland Tapir-Sorocaba 2', 'Lowland Tapir-Chuva F', 'Lowland Tapir-Sorocaba 5', 'Lowland Tapir-Edinha F', 'Lowland Tapir-Feminha F', 'Lowland Tapir-Pistolinha M', 'Lowland Tapir-Sorocaba', 'Lowland Tapir-Chuvisco M', 'Lowland Tapir-Riscado M'], 'Puma': ['Puma-M-Taz', 'Puma-M-Skit', 'Puma-M-Pops', 'Puma-M-Phoenix', 'Puma-M

The following code generates pairs of images - it pairs a base image (from test dataset) with one random image from every individual within the species of the base image. (This needs to modified in the future to extract one representative image from each individual instead of one random image.)

In [0]:
size = 2

def generate_test_pairs(size, train_data, test_data):
  image = train_data["Amur Tiger-261"][0]
  image = image[::size, ::size, ::]
  dim1 = image.shape[0]
  dim2 = image.shape[1]

  pairs = []
  idx1 = 0
  idx2 = 0

  x_pair = np.zeros([3000, 2, dim1, dim2, 3])  # 2 is for pairs

  for ind in test_data.keys():
    species = ind[:ind.find("-")]
    individuals = species_dict[species]
    for footprint in test_data[ind]:
      for item in individuals:
        footprints = train_data[item]
        total_samples = len(footprints)
        sample = np.random.choice(range(total_samples), 1, replace=False) # Needs to replace this with representative image for each individual instead of random sampling
        sample_image = footprints[sample[0]]
        img1 = footprint
        img2 = sample_image
        # Reduce the size
        img1 = img1[::size, ::size, ::]
        img2 = img2[::size, ::size, ::]
        # Store the images to the initialized numpy array
        x_pair[idx1, 0, :, :, :] = img1
        x_pair[idx1, 1, :, :, :] = img2
        pairs.append((idx2, ind, item))
        idx1 += 1
      idx2 += 1

  x_pair_new = np.zeros([idx1, 2, dim1, dim2, 3])  # 2 is for pairs
  for counter in range(idx1):
      x_pair_new[counter, 0, :, :, :] = x_pair[counter, 0, :, :, :]
      x_pair_new[counter, 1, :, :, :] = x_pair[counter, 1, :, :, :]

  # Concatenate genuine pairs and imposite pairs to get the whole data
  X = np.concatenate([x_pair_new], axis=0)/255

  return pairs, X

In [0]:
real_test_pairs, x_real_test = generate_test_pairs(size, train_data, test_data)

In [0]:
real_predicted_values = trained_model.predict([x_real_test[:, 0], x_real_test[:, 1]])

In [0]:
results = np.concatenate([real_test_pairs, real_predicted_values], axis=1)

Then, the output values of the model are compared for every test image to select the individual that has the lowest value. It is assumed that the value with the lowest value represents the class of individual that the test image most likely belongs to.

In [66]:
results_dict = {}

for image, actual_ind, test_ind, score in results:
  if image not in results_dict:
    results_dict[image] = [actual_ind, test_ind, score]
  existing_score = results_dict[image][2]
  if score < existing_score:
    results_dict[image] = [actual_ind, test_ind, score]

print(results_dict)

{'0': ['Amur Tiger-237', 'Amur Tiger-682', '0.29614305'], '1': ['Amur Tiger-261', 'Amur Tiger-682', '0.017769689'], '2': ['Amur Tiger-261', 'Amur Tiger-261', '0.05716519'], '3': ['Amur Tiger-279', 'Amur Tiger-1020', '0.028992444'], '4': ['Amur Tiger-440', 'Amur Tiger-682', '0.01969221'], '5': ['Amur Tiger-440', 'Amur Tiger-440', '0.19495362'], '6': ['Amur Tiger-565', 'Amur Tiger-565', '0.05082969'], '7': ['Amur Tiger-682', 'Amur Tiger-682', '0.61025023'], '8': ['Amur Tiger-682', 'Amur Tiger-565', '0.12156996'], '9': ['Amur Tiger-1020', 'Amur Tiger-565', '0.02764908'], '10': ['Amur Tiger-1020', 'Amur Tiger-279', '0.091716945'], '11': ['Bengal Tiger-Aria', 'Bengal Tiger-Mona', '0.18202755'], '12': ['Bengal Tiger-Aria', 'Bengal Tiger-India', '0.28897348'], '13': ['Bengal Tiger-Aria', 'Bengal Tiger-Moki', '0.22625233'], '14': ['Bengal Tiger-Fenimore', 'Bengal Tiger-Moki', '0.0055234483'], '15': ['Bengal Tiger-Fenimore', 'Bengal Tiger-Rajah', '0.16140138'], '16': ['Bengal Tiger-India', 'Ben

Finally, the accuracy is measured by comparing the actual individual classes with the predicted individual classes.

In [67]:
total = 0
correct = 0

for img in results_dict.keys():
  total += 1
  actual = results_dict[img][0]
  predicted = results_dict[img][1]
  if actual == predicted:
    correct += 1

print("Total Images = ", total)
print("Number of Correct Predictions = ", correct)
print("Accuracy = ", correct/total)

Total Images =  204
Number of Correct Predictions =  36
Accuracy =  0.17647058823529413
